In [80]:
import env
import os
import pandas as pd
from sklearn.model_selection import train_test_split

# get data

In [17]:
query = '''select bedroomcnt,bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, fips, propertylandusetypeid from properties_2017
WHERE propertylandusetypeid = 261;'''

In [10]:
def get_connection(db, user=env.user, host=env.host, password=env.password):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

In [12]:
url = get_connection('zillow')

In [63]:
prop17 = pd.read_sql(query,url)

In [64]:
prop17.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips,propertylandusetypeid
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0,261.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0,261.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0,261.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0,261.0
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0,261.0


# cache data

In [23]:
def cache_dataframe(df, file_name):
    '''
    Cache a Pandas DataFrame as a CSV file in the current working directory
    
    Parameters:
    df (pandas.DataFrame): The DataFrame to cache
    file_name (str): The name of the CSV file
    
    Returns:
    None
    '''
    
    df.to_csv(file_name, index=False)

In [24]:
cache_dataframe(prop17,'prop17.csv')

# check it out

In [67]:
prop17.shape

(2152863, 8)

In [62]:
prop17.dtypes

dtype('int32')

In [29]:
prop17.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips,propertylandusetypeid
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0,261.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0,261.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0,261.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0,261.0
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0,261.0


In [68]:
prop17.isna().sum()

bedroomcnt                        11
bathroomcnt                       11
calculatedfinishedsquarefeet    8484
taxvaluedollarcnt                493
yearbuilt                       9337
taxamount                       4442
fips                               0
propertylandusetypeid              0
dtype: int64

In [72]:
(prop17.yearbuilt == prop17.yearbuilt.astype(int)).value_counts()

True    2140235
Name: yearbuilt, dtype: int64

In [73]:
(prop17.calculatedfinishedsquarefeet == prop17.calculatedfinishedsquarefeet.astype(int)).value_counts()

True    2140235
Name: calculatedfinishedsquarefeet, dtype: int64

In [74]:
(prop17.taxvaluedollarcnt == prop17.taxvaluedollarcnt.astype(int)).value_counts()

True    2140235
Name: taxvaluedollarcnt, dtype: int64

In [75]:
(prop17.taxamount == prop17.taxamount.astype(int)).value_counts()

False    2111620
True       28615
Name: taxamount, dtype: int64

In [76]:
(prop17.fips == prop17.fips.astype(int)).value_counts()

True    2140235
Name: fips, dtype: int64

# clean data

In [70]:
prop17 = prop17.drop(columns= 'propertylandusetypeid')

In [71]:
prop17 = prop17.dropna()

In [78]:
prop17['yearbuilt'] = prop17.yearbuilt.astype(int)
prop17['calculatedfinishedsquarefeet'] = prop17.yearbuilt.astype(int)
prop17['taxvaluedollarcnt'] = prop17.taxvaluedollarcnt.astype(int)
prop17['fips'] = prop17.fips.astype(int)

In [ ]:
prop17.shape

In [43]:
2140235/ 2152863

0.9941343225277224

In [85]:
def split_dataframe(df, train_size=0.6, val_size=0.2, test_size=0.2, random_state=None):
    '''
    Split a Pandas DataFrame into train, validation, and test sets

    Parameters:
    df (pandas.DataFrame): The DataFrame to split
    train_size (float): The proportion of the data to use for training (default=0.6)
    val_size (float): The proportion of the data to use for validation (default=0.2)
    test_size (float): The proportion of the data to use for testing (default=0.2)
    random_state (int): The random seed to use for the train/test split (default=None)

    Returns:
    train_df (pandas.DataFrame): The training set
    val_df (pandas.DataFrame): The validation set
    test_df (pandas.DataFrame): The test set
    '''

    # Split the data into train and test sets
    train, test = train_test_split(df, test_size=test_size, random_state=random_state)

    # Calculate the proportion of the original data to allocate to validation
    val_prop = val_size / (train_size + val_size)

    # Split the remaining data into train and validation sets
    train, val = train_test_split(train, test_size=val_prop, random_state=random_state)

    return train, val, test

In [87]:
train, val, test = split_dataframe(prop17,train_size=0.6, val_size=0.2, test_size=0.2, random_state=42)

In [88]:
train

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
100475,3.0,2.0,1935,114686,1935,1536.35,6037
1891683,3.0,2.0,1962,226635,1962,2407.74,6059
985105,3.0,2.0,1950,538475,1950,6613.79,6037
119764,4.0,2.0,1955,95010,1955,1336.49,6037
388552,3.0,2.0,1954,580000,1954,6971.24,6037
...,...,...,...,...,...,...,...
109743,3.0,2.0,1959,49662,1959,1171.56,6037
1547574,3.0,1.0,1948,218641,1948,2706.94,6059
1804611,4.0,2.0,1955,304021,1955,3682.35,6037
37983,3.0,2.0,1950,777700,1950,9217.66,6037
